# Support Vector Regression

如何結合 L2-Regularized Linear Model 與 Kernel, 獲得 Analytic Solution for kernel ridge regression?

for any L2-regularized linear model

$$
\min_w \frac{\lambda}{N} w^T w + \frac{1}{N} \sum_{n=1}^N err \big( y_n, w^T z_n \big)
$$

optimal $ w_{*} = \sum_{n=1}^N \beta_n z_n $

for Regression with Squared Error:

$$
err \big( y_n, w^T z_n \big) = \big( y - w^T z \big)^2
$$

## Kernel Ridge Regression Problem

$
\min_w \frac{\lambda}{N} w^T w + \frac{1}{N} \sum_{n=1}^N \big( y_n - w^T z_n \big)^2
$

optimal $ w_{*} = \sum_{n=1}^N \beta_n z_n $

將 w 替換成 $ \beta $, 就可以將問題變成 $ \beta $ 的最佳化。 

$$
\min_{\beta} \frac{\lambda}{N} \sum_{n=1}^N \sum_{m=1}^N \beta_n \beta_m K(x_n, x_m) 
+ \frac{1}{N} \sum_{n=1}^N \Big( y_n - \sum_{m=1}^N \beta_m  K(x_n,x_m) \Big)^2 \\
= \frac{\lambda}{N} \beta^T K \beta + \frac{1}{N} \big( \beta^T K ^T K \beta - 2 \beta^T K^T y + y^T y \big)
$$

### Solving Kernel Ridge Regression

$
E_{aug}(\beta) = \frac{\lambda}{N} \beta^T K \beta + \frac{1}{N} \big( \beta^T K ^T K \beta - 2 \beta^T K^T y + y^T y \big)
$

$
\nabla E_{aug}(\beta) = \frac{2}{N} \big( \lambda K^T I \beta + K^T K \beta - k^T y \big) \\
= \frac{2}{N} K^T \big( ( \lambda I + K ) \beta - y \big)
$

want $ \nabla E_{aug} (\beta) = 0 $ : one anayttic solution

$$
\beta = \big( \lambda I + K \big)^{-1} y
$$

- $ (...)^{-1} $ always exists for $ \lambda \gt 0 $, because K is positive semi-definite. (Mercer's condition)
- time complexity: $ O(N^3) $ with simple dense matrix inversion

Can now do non-linear regression 'easily'.

![img](imgs/c206-linear-kernel-ridge-reg.jpg)